In [3]:
import sys
import os
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path + "/src/simulations_v2")
from load_params import load_multigroup_params, load_params
from multi_group_simulation import MultiGroupSimulation

In [6]:
num_trajectories = 200

interaction_matrix = np.array([[8,0.1,0.1,1,0.05,0.05,0.2],
                               [0.22,4,0.1,1.2,0.05,0.2,1.8],
                               [0.17,0.07,9,1,0.05,0.05,0.2],
                               [2.26,1.22,1.37,1,0.15,0.3,1.56],
                               [0.21,0.1,0.13,0.28,1.8,0.2,1.56],
                               [0.09,0.15,0.05,0.23,0.08,1.8,1.56],
                               [0.026,0.106,0.016,0.091,0.048,0.12,3.5]])

group_names = ['UG (off campus)', 'GS (research)', 'GS (other)', 'Faculty/Staff (student facing)', 'Faculty/Staff (non student facing)', 'Faculty/Staff (off campus)', 'Ithaca Community']

num_groups = len(group_names)
free_and_infectious_local = np.zeros((num_trajectories, num_groups))
QS_local = np.zeros((num_trajectories, num_groups))
QI_local = np.zeros((num_trajectories, num_groups))
    
ug_off_campus_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/nominal_local_only_option_4/ug_off_campus.yaml')[1]
gs_research_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/nominal_local_only_option_4/grad_research.yaml')[1]
gs_other_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/nominal_local_only_option_4/grad_other.yaml')[1]
faculty_staff_student_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/nominal_local_only_option_4/faculty_staff_student_same_age.yaml')[1]
faculty_staff_non_student_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/nominal_local_only_option_4/faculty_staff_non_student_same_age.yaml')[1]
faculty_staff_off_campus_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/nominal_local_only_option_4/faculty_staff_off_campus_same_age.yaml')[1]
ithaca_community_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/nominal_local_only_option_4/ithaca_community.yaml')[1]

params_list = [ug_off_campus_params.copy(), gs_research_params.copy(), gs_other_params.copy(), faculty_staff_student_params.copy(), faculty_staff_non_student_params.copy(), faculty_staff_off_campus_params.copy(), ithaca_community_params.copy()]

for index, params in enumerate(params_list):
    params['expected_contacts_per_day'] = interaction_matrix[index, index]

sim = MultiGroupSimulation(params_list, interaction_matrix, group_names)
    
for i in range(num_trajectories):
    sim.reset_initial_state()
    for _ in range(18):
        sim.step()

    for j in range(num_groups):
        free_and_infectious_local[i, j] = sim.get_free_infectious(j)
        QS_local[i, j] = sim.sims[j].QS
        QI_local[i, j] = sim.sims[j].QI

print("No return: ")
print("free-and-infectious: ", np.mean(free_and_infectious_local, axis=0))

populations = [sim.sims[j].pop_size for j in range(num_groups)]
print("prevalence:", np.mean(free_and_infectious_local, axis=0) / populations)
print("QS: ", np.mean(QS_local, axis=0))
print("QI: ", np.mean(QI_local, axis=0))

No return: 
free-and-infectious:  [ 0.565  1.61   0.785  1.81   0.745  2.665 19.4  ]
prevalence: [0.00050222 0.00096005 0.00061089 0.00050306 0.00039067 0.00055776
 0.0003129 ]
QS:  [  5.4   10.05   5.61  18.36   6.09  21.36 219.03]
QI:  [ 3.885  4.745  4.195  9.765  2.71   5.38  60.425]


In [10]:
# statistics within Cornell (including students, faculty / staff)
print(np.sum(np.mean(free_and_infectious_local[:, :6], axis=0)))
print(np.sum(np.mean(QI_local[:, :6], axis=0)))
print(np.sum(np.mean(QS_local[:, :6], axis=0)))

8.18
30.68
66.87


In [8]:
# quarantine/isolation statistics of students
print(np.sum(np.mean(QI_local[:, :3], axis=0)))
print(np.sum(np.mean(QS_local[:, :3], axis=0)))

12.825
21.060000000000002


In [9]:
print(np.sum(np.percentile(QI_local[:, :3], 90, axis=0)))
print(np.sum(np.percentile(QS_local[:, :3], 90, axis=0)))

21.0
42.0
